In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# get raw data
df = pd.read_csv("raw_data/Machine Learning Answers.md", sep="\n")

In [3]:
df["## Machine Learning"].str[0:5].value_counts()

- [ ]    287
####      96
- [x]     96
[Refe      7
**Exp      7
![Mac      6
[Sour      3
![ima      3
  **E      1
  `Un      1
`The       1
`that      1
`// t      1
  `//      1
`//Yo      1
Name: ## Machine Learning, dtype: int64

In [4]:
# clean df of explanations, references
df = df[df["## Machine Learning"].str[0:5].isin(["- [ ]",
                                                 "#### ",
                                                 "- [x]",
                                                 "![Mac",
                                                 "![ima"])]

In [5]:
lst = list(df["## Machine Learning"])

In [6]:
# split list
list_of_lists = []
tmp_list = []
for element in lst:
    if element[0] == "#":
        list_of_lists.append(tmp_list)
        tmp_list = []
    tmp_list.append(element)
    if element == lst[-1]:
        list_of_lists.append(tmp_list)
del list_of_lists[0]

In [7]:
# min max of length
d = {}
for i in range(len(list_of_lists)):
    d[i] = len(list_of_lists[i])
print(min(d, key=d.get))
print(max(d, key=d.get))

64
9


In [8]:
list_of_lists[65]

['#### Q66. The activations for class A, B and C before softmax were 10, 8 and 3. The difference in softmax values for class A and class B would be :',
 '![image](images/machine-learning_Q62.png)',
 '- [x] 76%',
 '- [ ] 88%',
 '- [ ] 12%',
 '- [ ] 0.0008%']

In [9]:
# put list elements in order: Q, ans(4), correct answ, image
for q in list_of_lists:
    # if only 3 answers, add 4th
    if len(q) == 4:
        q += [""]
    # extract image link, append at end
    if q[1][0] == "!":
        q.append(re.search(r"images\/.+\.(jpg|png)", q[1]).group(0))
        del q[1]
    else:
        q += [""]
    # check correct answer, insert index at position 5
    for i in range(1,5):
        try:
            if q[i][3] == "x":
                q.insert(5, i)
        except:
            continue

In [10]:
# turn to DataFrame
res = pd.DataFrame(list_of_lists,
                   index=list(range(len(list_of_lists))),
                   columns=["question", "answ1", "answ2", "answ3", "answ4", "correct", "image"])

In [11]:
# Clean answers
res["answ1"] = res["answ1"].str[5:]
res["answ2"] = res["answ2"].str[5:]
res["answ3"] = res["answ3"].str[5:]
res["answ4"] = res["answ4"].str[5:]
# Clean question
res["question"] = res["question"].str.lstrip("#### ")

In [12]:
res.tail(2)

,question,answ1,answ2,answ3,answ4,correct,image
94,Q95. When is a decision tree most commonly used?,with big data products,for supervised machine learning binary classi...,to find thd best data cluster,"to determine ""Q"" in Q-learning reinforcement ...",2,
95,Q96. An organisation that owns dozens of shopp...,most machine learning models are not designed...,it might be unethical for the business to ide...,it will be difficult to decide between superv...,the image in the video would not be high qual...,2,


In [13]:
# save csv
res.to_csv("data/results.csv", index=False)